In [ ]:
import pickle

In [ ]:
#with open('place_id_filenames_map.pkl','wb') as fh: pickle.dump(place_id_filenames_map, fh)
with open('place_id_filenames_map.pkl','rb') as fh: place_id_filename_map = pickle.load(fh)

In [ ]:
len(place_id_filename_map)

In [ ]:
import tabula
import tqdm

import os
import pandas as pd
import numpy as np

In [ ]:
dir='downloads'

In [ ]:
df = pd.read_csv('facility_name_id_filename.csv',index_col=0)
df.head()

In [ ]:
import pandas as pd
from io import StringIO

match_dmg_str = r'\d+[°|o] \d+’ \d+”'
match_dmg_str_commas = r'\d+[°|o][,]? \d+’[,]? \d+” [N|W]?'
match_deg_str = r'[+-]?\d+[.]\d*[°|o]'

def find_pages_after_match(pages, match):
    for df in pages:
        cols = df.columns
        df = df.astype(str)
        for cname in cols:
            if df[cname].str.contains(match).any():
                yield df

def find_row_with_match(page, matches):
    cols = page.columns
    dfall = page.astype(str).apply(lambda x: "-".join(x), axis =1)
    for match in matches:
        if not dfall.empty and dfall.str.contains(match).any():
            return dfall.dropna().loc[dfall.dropna().str.contains(match)].values[0]
                
def get_value_with_match(pages, matches):
    for page in pages:
        find_row_with_match(page, matches)

import re
dmg = re.compile(match_dmg_str)
dmg_parts = re.compile(r'(\d+)[°|o] (\d+)’ (\d+)”')
degdecimal = re.compile(match_deg_str)
degdecimal_parts = re.compile(r'([+-]?\d+[.]\d*)[°|o]')
dmg_commas = re.compile(match_dmg_str_commas)
dmg_commas_parts = re.compile(r'(\d+)[°|o][,]? (\d+)’[,]? (\d+)” [N|W]?')

def parse_degdecimal(x):
    lat, lon = dmgdecimal.findall(x)
    
def parse_dmg(x):
    _,d,m,s,_=dmg_parts.split(x)
    return float(d)+float(m)/60+float(s)/3600

def parse_dmg_commas(x):
    _,d,m,s,_=dmg_commas_parts.split(x)
    return float(d)+float(m)/60+float(s)/3600    

def parse_deg_decimal(x):
    _,y,_=degdecimal_parts.split(x)
    return float(y)

def parse_lat_lon(x):
    try:
        lat, lon = dmg.findall(x)
        lat = parse_dmg(lat)
        lon = parse_dmg(lon)
        lon = -lon
    except:
        try:
            lat, lon = dmg_commas.findall(x)
            lat = parse_dmg_commas(lat)
            lon = parse_dmg_commas(lon)
            lon = -lon
        except:
            lat, lon = degdecimal.findall(x)
            lat = parse_deg_decimal(lat)
            lon = parse_deg_decimal(lon)
    return lat,lon

# Keep only active facilities, i.e. for which NPDES permit was downloaded (only active ones have been downloaded)

In [ ]:
records = [(k,a) for k in place_id_filename_map for a in place_id_filename_map[k]]

In [ ]:
len(place_id_filename_map)

In [ ]:
df_facility_files = pd.DataFrame.from_records(records, columns=['facility_place_id','filename'])
df_facility_files.head()

In [ ]:
dfm = df.merge(df_facility_files, on='facility_place_id', suffixes=('_l','_r')).drop(['filename_l'], axis=1).rename({'filename_r':'filename'}, axis=1)
dfm.head()

In [ ]:
def get_ll_for_facility_name(facility_name):
    for fname in dfm.query(f'facility_name.str.startswith("{facility_name}")').filename.values:
        try:
            print('Trying ',fname)
            pages = tabula.read_pdf(f'{dir}/{fname}', pages='all', guess=False)
            pagesafter = find_pages_after_match(pages,'Table 2')
            page0 = next(pagesafter)
            page1 = next(pagesafter)
            row_match = find_row_with_match(page0, [match_dmg_str, match_deg_str, match_dmg_str_commas])
            print('match value: ', row_match)
            print('match value: ', row_match)
            lat, lon = parse_lat_lon(row_match)
            print(facility_name, lat,lon)
            return (lat,lon)
            break; # goto the next facility
        except:
            continue
    return None

In [ ]:
get_ll_for_facility_name('Aerojet Interim GW Extraction & Treatment System')

In [ ]:
pages = tabula.read_pdf(f'{dir}/r5-2020-0051-001_npdes.pdf', pages='all', guess=False)
#pages

In [ ]:
pagesafter = find_pages_after_match(pages,'Table 2')
page0 = next(pagesafter)
page1 = next(pagesafter)

In [ ]:
page0

In [ ]:
row_match = find_row_with_match(page0, [match_dmg_str, match_deg_str, match_dmg_str_commas])
row_match

parse_lat_lon(row_match)

In [ ]:
ll = {}
failures = {}
for i,r in df.iterrows():
    facility_name = r['facility_name']
    latlon = get_ll_for_facility_name(facility_name)
    if latlon == None:
        print('Failed for ', facility_name)
        failures[facility_name] = None
        if len(failures) > 5: break;
    else:
        ll[facility_name]=latlon

In [ ]:
dfm.query('facility_name == "Aerojet Sacramento Facility"')

In [ ]:
pages = tabula.read_pdf(f'downloads/r5-2013-0156_npdes.pdf', pages='all', guess=False)


In [ ]:
for i, r in pages[0].iterrows():
    print(r.values)

In [ ]:
len(ll)

In [ ]:
dfll = pd.DataFrame.from_records([(k,ll[k][0],ll[k][1]) for k in ll],columns=['facility_name','latitude','longitude'])

In [ ]:
len(dfll)

In [ ]:
len(dfm.facility_name.unique())

In [ ]:
dfmll = dfm.merge(dfll, on='facility_name', how='outer')
dfmll

In [ ]:
dfmx = dfmll.query('latitude.isna()')[dfm.columns]

In [ ]:
dfmx

In [ ]:
#facility_name = "Sacramento Regional WWTP"
#facility_name = "American Canyon Water Reclamation Facility (WRF) (formerly WWTP)"

dfm.query(f'facility_name.str.startswith("{facility_name}")')

In [ ]:
fname = dfm.query(f'facility_name.str.startswith("{facility_name}")').filename.values[0]
fname

In [ ]:
pages = tabula.read_pdf(f'{dir}/{fname}', pages='all', guess=False)#columns=['date','time','remarks'])

In [ ]:
pages

In [ ]:
get_value_with_match(pages, [match_dmg_str, match_deg_str])

In [ ]:
lat, lon = parse_lat_lon(get_value_with_match(pages, [match_dmg_str, match_deg_str]))
lat, lon